In [1]:
import tomographer
#import logging
#tomographer.cxxlogger.level = logging.NOTSET # slow things down a bit, to test for progress

In [2]:
# Give us a nice progress bar
import jpyprogress

In [4]:
import numpy as np

Nm = np.array([ 490, 510, 494, 506, 980, 20 ])

Emn = [
    # X direction
    np.array([[1, 1],
              [1, 1]]),
    np.array([[1, -1],
              [-1, 1]]),
    # Y direction
    np.array([[1, -1j],
              [1j, 1]]),
    np.array([[1, 1j],
              [-1j, 1]]),
    # Z direction
    np.array([[1, 0],
              [0, 0]]),
    np.array([[0, 0],
              [0, 1]]),
    ]

rho_ref = np.array([[1,0],[0,0]])

r = None
with jpyprogress.NiceProgressBar("Running random walks for the tomography procedure ... ") as prg:
    def progress_fn(r):
        prg.progress(fraction_done=r.total_fraction_done)
        #print("{:.2%}: {} + ({}) ".format(r.total_fraction_done, r.num_completed,
        #                                ",".join(["{:.2%}".format(w.fraction_done) for w in r.workers if w]) ))
    r = tomographer.tomorun.tomorun(dim=2, Nm=Nm, Emn=Emn,
                                    hist_params=tomographer.UniformBinsHistogramParams(0.9,1,50),
                                    mhrw_params=tomographer.MHRWParams(0.03,35,500,32768),
                                    fig_of_merit="obs-value",
                                    num_repeats=12,
                                    observable=rho_ref,
                                    progress_fn=progress_fn)

#print(repr(r))
#print(r['final_histogram'].prettyPrint(100))
print(r['final_report'])



                                        Final Report of Runs                                        
----------------------------------------------------------------------------------------------------
	step size       = 0.03
	# iter. / sweep = 35
	# therm. sweeps = 500
	# run sweeps    = 32768
# 0: 0.9|                           ........-+x%##%o-...   |1                  [accept ratio = 0.20]
    error bars: 48 converged / 1 maybe (1 isolated) / 1 not converged
# 1: 0.9|                            .......-+x%##%o-...   |1                  [accept ratio = 0.20]
    error bars: 49 converged / 0 maybe (0 isolated) / 1 not converged
# 2: 0.9|                           ........-+o%##%o-....  |1                  [accept ratio = 0.20]
    error bars: 49 converged / 1 maybe (1 isolated) / 0 not converged
# 3: 0.9|                            .......-+x%##%o-...   |1                  [accept ratio = 0.20]
    error bars: 48 converged / 2 maybe (2 isolated) / 0 not converged
# 4: 0.9|          

In [5]:
print(r['final_histogram'].params.min, r['final_histogram'].params.max, r['final_histogram'].params.num_bins)
print(r['final_histogram'].bins)

0.9 1.0 50
[[  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  2.54313151e-06]
 [  0.00000000e+00]
 [  1.27156576e-05]
 [  3.05175781e-05]
 [  6.61214193e-05]
 [  1.90734863e-04]
 [  5.00996908e-04]
 [  1.38854980e-03]
 [  3.62141927e-03]
 [  8.22194417e-03]
 [  1.82215373e-02]
 [  3.64125570e-02]
 [  6.54017131e-02]
 [  1.04138692e-01]
 [  1.44983927e-01]
 [  1.74850464e-01]
 [  1.73166911e-01]
 [  1.38122559e-01]
 [  8.34935506e-02]
 [  3.58327230e-02]
 [  9.81140137e-03]
 [  1.42923991e-03]
 [  9.40958659e-05]
 [  5.08626302e-06]
 [  0.00000000e+00]
 [  0.000